# 네이버 영화평 감성 분석

In [1]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 경고 제거 
import warnings

warnings.filterwarnings(action='ignore')

In [3]:
# 네이버 영화 리뷰 데이터로 검색 : 누가 크롤링해서 라벨링 해 놓은 것 
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt",sep='\t')

train_df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
train_df.shape, test_df.shape

((150000, 3), (50000, 3))

### 1. 데이터 전처리 
- 트레인 데이터 셋의 경우 

In [5]:
# 중복 여부 확인 - number of unique를 세는 nunique. 15만개가 나와야 하는데 그렇지 않으므로 중복데이터 존재.
train_df.document.nunique()

146182

In [6]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'],inplace=True)
train_df.shape      # 위의 nunique값과 다른 개수가 나왔다. 추가 확인 필요

(146183, 3)

In [7]:
# null데이터 확인 - document에서 공백 데이터가 있는 것을 확인 
train_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [8]:
# null 제거
train_df = train_df.dropna(how='any')
train_df.shape

(146182, 3)

In [9]:
# 긍정 / 부정 레이블의 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- 테스트 데이터 셋의 경우 

In [10]:
test_df.shape

(50000, 3)

In [11]:
# 중복 여부 확인
test_df.document.nunique()

49157

In [12]:
# 중복 제거 
test_df.drop_duplicates(subset=['document'],inplace=True)
test_df.shape

(49158, 3)

In [13]:
# Null데이터 확인
test_df.isna().sum()

id          0
document    1
label       0
dtype: int64

In [14]:
# null 제거
test_df = test_df.dropna(how='any')
test_df.shape

(49157, 3)

In [15]:
# 긍정 / 부정 레이블의 분포
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

### 2. 텍스트 전처리 

In [16]:
# 한글과 공백 이외에는 제거 
# 한글 정규표현식 -> ㄱ - ㅎ : 자음, ㅏ - ㅣ : 모음, 가 - 힣 : 단어
train_df['document'] = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')

In [17]:
train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [18]:
# ''데이터는 Nan으로 변환한 후 제거 
train_df['document'].replace('',np.nan, inplace=True)
train_df.document.isnull().sum()

391

In [19]:
train_df = train_df.dropna(how='any')
train_df.shape

(145791, 3)

- test dataset 

In [20]:
test_df['document'] = test_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
test_df['document'].replace('',np.nan, inplace=True)
test_df.document.isnull().sum()

162

In [21]:
test_df = test_df.dropna(how='any')
test_df.shape

(48995, 3)

- 저장 

In [22]:
train_df.to_csv('naver_movie_train.csv',sep='\t',index=False)
test_df.to_csv('naver_movie_test.csv',sep='\t',index=False)

### 3. 한글 처리

In [23]:
from konlpy.tag import Okt
okt = Okt()

In [24]:
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','ㅋㅋ','ㅎㅎ','ㅠㅠ']

In [25]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
okt.morphs(text)

['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정']

In [26]:
okt.morphs(text,stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [27]:
from tqdm import tqdm_notebook

X_train = []
for sentence in tqdm_notebook(train_df.document):
    morphs = okt.morphs(sentence,stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stopwords])
    X_train.append(temp_X)

In [28]:
X_test = []
for sentence in tqdm_notebook(test_df.document):
    morphs = okt.morphs(sentence,stem=True)
    temp_X = ' '.join([word for word in morphs if not word in stopwords])
    X_test.append(temp_X)

In [29]:
y_train = train_df.label.values     #label까지만 하면 시리즈, values까지 가면 array
y_test = test_df.label.values

### 4. Feature 변환, 모델 학습 / 예측 / 평가 

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [31]:
cvect = CountVectorizer()

cvect.fit(X_train)

X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)

In [32]:
lr = LogisticRegression()
lr.fit(X_train_cv,y_train)

LogisticRegression()

In [33]:
pred = lr.predict(X_test_cv)
accuracy_score(y_test,pred)

0.8253699357077253

### 5. 실제 테스트

In [34]:
review1 = "재미있네요 조커와는 다른 매력적인 악당입니다. 조커는 밑바닥으로 추락하면서 빌런이 되지만 크루엘라는 정상을 찍으며 빌런이 됩니다."
review2 = "영화보면서 존거 처음입니다.!! 미국식 양산형범죄영화 느낌이였음개그는 하나도 안웃기고 크루엘라의 컨셉도 몰입이 안됨"

In [35]:
review1 = review1.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
review2 = review2.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')

In [36]:
review1             
# 안바뀜

'재미있네요 조커와는 다른 매력적인 악당입니다. 조커는 밑바닥으로 추락하면서 빌런이 되지만 크루엘라는 정상을 찍으며 빌런이 됩니다.'

In [37]:
# 다른방법 - re를 사용
import re

review1 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','',review1)
print(review1)

재미있네요 조커와는 다른 매력적인 악당입니다 조커는 밑바닥으로 추락하면서 빌런이 되지만 크루엘라는 정상을 찍으며 빌런이 됩니다


- review 1

In [38]:
morphs = okt.morphs(review1)
review = ' '.join([word for word in morphs if not word in stopwords])
review

'재미있네요 조커 와는 다른 매력 적 인 악당 입니다 조커 밑바닥 추락 하면서 빌런 되지만 크루 엘라 정상 찍으며 빌런 됩니다'

In [39]:
review_cv = cvect.transform([review])
pred = lr.predict(review_cv)
pred[0]

1

- review2

In [40]:
review2 = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','',review2)
print(review2)

영화보면서 존거 처음입니다 미국식 양산형범죄영화 느낌이였음개그는 하나도 안웃기고 크루엘라의 컨셉도 몰입이 안됨


In [41]:
morphs = okt.morphs(review2)
review = ' '.join([word for word in morphs if not word in stopwords])
review_cv = cvect.transform([review])
pred = lr.predict(review_cv)
pred[0]

0

- Case 2

In [42]:
reviews = ['아름다운 음악과 아름다운 풍광~ 그렇지 못한 현실이 찡하네요.','메세지와 작위성의 불협화음!!!!']

In [43]:
for review in reviews:
    review = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','',review)
    morphs = okt.morphs(review)
    review = ' '.join([word for word in morphs if not word in stopwords])
    review_cv = cvect.transform([review])
    pred = lr.predict(review_cv)
    print(f'"{review}"의 분석 결과는 {pred[0]}')

"아름다운 음악 아름다운 풍 광 그렇지 못 현실 찡하네요"의 분석 결과는 1
"메세지 작위 성의 불협화음"의 분석 결과는 0


### 6. GridSearchCV로 최적 파라미터 찾기 

In [44]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [45]:
pipeline = Pipeline([
    ('cvect' ,CountVectorizer()),
    ('lr', LogisticRegression())
])

params = {
    "cvect__ngram_range" : [(1,1),(1,2)],
    "cvect__max_df" : [0.9,0.99],
    "cvect__min_df" : [1,3],
    "lr__C" : [1,5]
}

In [46]:
grid_pipe = GridSearchCV(
    pipeline,param_grid = params, cv=3, scoring = 'accuracy',n_jobs=-1
)

%time grid_pipe.fit(X_train,y_train)
print(grid_pipe.best_score_, grid_pipe.best_params_)

Wall time: 1min 51s
0.840538853564349 {'cvect__max_df': 0.9, 'cvect__min_df': 1, 'cvect__ngram_range': (1, 2), 'lr__C': 1}


In [47]:
pred = grid_pipe.best_estimator_.predict(X_test)
acc = accuracy_score(pred,y_test)
print(f'CountVectorizer + LogisticRegression 정확도 :{acc:.4f}')

CountVectorizer + LogisticRegression 정확도 :0.8462
